In [36]:
import sys
import pandas as pd
import numpy as np

from keras import models
from keras import layers
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout,ZeroPadding2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
import tarfile
import zipfile
import urllib
import urllib.request
import hashlib
import requests
import sys
import requests
import cv2
import glob
import re 




## Helper Functions 
Helper functions , Tar extract , Resize files 

In [37]:
################ Helper Function #######################
#            ExtractTarFile      ######################
#  Extract Tar all files (regerdless of the directory)#
#  Structure from a Tar file                          #
#  Paramters :                                        #
#  FileName - The Tar file name                       #
#  TargetDirectory - The target to place the          #
#  extracted files                                    #
#  DeleteAtEnd - If true delete the Tar file          #


def ExtractTarFile(FileName,TargetDirectory,DeleteAtEnd):
    #Try to create the directory (in case not exist)
    if(os.path.exists(TargetDirectory)==False):
        os.mkdir(TargetDirectory)
    #Iterate over the Tar file and Extract all files 
    with tarfile.open(FileName, 'r:tar') as _tar:
        for member in _tar:
            if member.isdir():
                continue
            fname = member.name.rsplit('/',1)[1]
            _tar.makefile(member, TargetDirectory + '/' + fname)
    #Delete Tr file (if DeletAtEnd is True )
    if (DeleteAtEnd):
        os.remove(FileName)

In [67]:
################ Helper Function #######################
#            BatchResizeImages     #####################
#  Resize All Jpg file in the directory               #
#  Structure from a Tar file                          #
#  Paramters :                                        #
#  SourceDirectory - Source Directory                 #
#  TargetDirectory - The target to place the          #
#  extracted files                                    #
#  DeleteAtEnd - If true delete the Tar file          #
#  width  - Target Image width                        #
#  height  - Target Image height                      #


def BatchResizeImages(SourceDirectory,TargetDirectory,DeleteAtEnd,width,height):


    cwd = os.getcwd()
    os.chdir(SourceDirectory)

    # Get images
    imgs = glob.glob('*.jpg')
    imgs.extend(glob.glob('*.jpeg'))

    

    #print('Resizing all images be %d pixels wide' % width)

    folder = TargetDirectory
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Iterate through resizing and saving
    for img in imgs:
        pic = cv2.imread(img, cv2.IMREAD_UNCHANGED)
        if pic is not None:
        #height = int(width * pic.shape[0] / pic.shape[1])
            pic = cv2.resize(pic, (width, height))
            cv2.imwrite(folder + '/' + img, pic)
            if (DeleteAtEnd):
                os.remove(img)
    
    #Go back to the original directory 
    os.chdir(cwd)

In [39]:
################ Helper Function #######################
#            Zip Files             #####################
#  Zip all files in a directory                       #
#                                                     #
#  Paramters :                                        #
#  folder_path - Source Directory                    #
#  output_path - The target zip file                 #
#                                                    #


def zip_folder(folder_path, output_path):
    
    parent_folder = os.path.dirname(folder_path)
    # Retrieve the paths of the folder contents.
    contents = os.walk(folder_path)
   
    zip_file = zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED)
    for root, folders, files in contents:
            # Include all subfolders, including empty ones.
            for folder_name in folders:
                absolute_path = os.path.join(root, folder_name)
                relative_path = absolute_path.replace(parent_folder + '\\',
                                                      '')
                #print ("Adding '%s' to archive." % absolute_path)
                zip_file.write(absolute_path, relative_path)
            for file_name in files:
                absolute_path = os.path.join(root, file_name)
                relative_path = absolute_path.replace(parent_folder + '\\',
                                                      '')
                #print ("Adding '%s' to archive." % absolute_path)
                zip_file.write(absolute_path, relative_path)
    #print("'%s' created successfully." % output_path)
    zip_file.close()


## Download image files 

In [40]:
#Prepare All URL location from 000 to 500
Base_Url= 'https://s3.amazonaws.com/google-landmark/train/images_XXX.tar'
Base_Target = 'images_XXX.tar'
URL_List = []
Target_List = []
for i in  range(500):
    counter  = (str(format(i, '0003d')))
    URL_List.append(re.sub('XXX', counter, Base_Url))
    Target_List.append(re.sub('XXX', counter, Base_Target))


In [ ]:
Start_Index = 1
Stop_Index = 50
#Base_Dir = "E:/Elements/kaggle/GoogleLandmarkRecognition2019/X.resized"
Base_Dir = "E:/X.resized"
for i in  range(Start_Index,Stop_Index):
    #Download Tar 
    print(URL_List[i])
    print(Target_List[i])
    print("downloading  file",i)
    myfile = requests.get(URL_List[i],verify=False, timeout=10)
    open(Target_List[i], 'wb').write(myfile.content)
    #Extract Tar 
    #ExtractTarFile(Target_List[i],"E:/Elements/kaggle/GoogleLandmarkRecognition2019",DeleteAtEnd=True)
    ExtractTarFile(Target_List[i],"E:/",DeleteAtEnd=True)
    Target = re.sub('X', str(i), Base_Dir)
    #Resize 
    #BatchResizeImages("E:/Elements/kaggle/GoogleLandmarkRecognition2019",Target,True,256,256)
    BatchResizeImages("E:/",Target,True,256,256)

In [68]:
 BatchResizeImages("c:/test","c:/testResize128",False,128,128)

## Create Zip File 

In [64]:
#!/usr/bin/env python
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))


zipf = zipfile.ZipFile('c:/temp/image.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('c:/temp', zipf)
zipf.close()

In [63]:
def fcount(path, map = {}):
  count = 0
  for f in os.listdir(path):
    child = os.path.join(path, f)
    if os.path.isdir(child):
      child_count = fcount(child, map)
      count += child_count + 1 # unless include self
  map[path] = count
  return count

path = "C:/temp/train_images_model"
map = {}
print (fcount(path, map))

2000


In [62]:
BatchResizeImages("C:/temp/train_images_model/2545/","C:/temp/train_images_model/2545/",False,128,128)